In [ ]:
import gitlab
from dotenv import load_dotenv
load_dotenv("./.env", override=True)
import os

In [ ]:
print(os.getenv("GIT_URL"))

In [ ]:
username = os.getenv("GIT_USERNAME")
git_url = os.getenv("GIT_URL")
gl = gitlab.Gitlab(git_url, os.getenv("GIT_TOKEN"))


In [ ]:
project_list=[]
# list all the projects
projects = gl.projects.list(iterator=True, owner=True)
for project in projects:
    project_list.append(project)

In [ ]:
print(len(project_list))

In [ ]:
commits = []
total_commits = 0
for project in project_list:
    project_id = project.id
    try:
        commits = project.commits.list(all=True) 
        user_commits = [commit for commit in commits if commit.author_email == os.getenv("GIT_EMAIL")]  
        total_commits += len(user_commits) 
    except:
        pass 



In [ ]:
print(total_commits)

In [ ]:
# Get the user ID by username
user = gl.users.list(username=os.getenv("GIT_USERNAME"))[0]
user_name = user.name
user_id = user.id
print(f"Checking for user {username} (#{user_id})")

total_commits = 0
i = 0
while i < len(projects):
    project = gl.projects.get(projects[i].id)
    print(f"Checking project {i}/{len(projects)} : {project.name}...")
    # Filter commits by author ID
    default_branch = project.default_branch
    project_branches = project.branches.list(get_all=True)

    for branch in project_branches:
        branch_name = branch.name
        branch_commits = project.commits.list(ref_name=branch.name, get_all=True)
        for commit in branch_commits:
            total_commits += (
                1
                if commit.author_name == user_name or commit.author_name == username
                else 0
            )
    i += 1

print(f"Total number of commits for {username} : {total_commits}")